In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
import pandas as pd
from fastdtw import fastdtw

In [21]:
df1 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/cpu_usage_2022_03_25_35_15m.csv')
#df2 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/mem_usage_2022_04_27_2_15m.csv')
#df3 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/api_server_count_2022_04_27_2_15m.csv')

In [22]:
df1.timestamp = pd.to_datetime(df1.timestamp, infer_datetime_format=True)  

In [23]:
cluster_id_list = df1['cluster_id'].to_list()

In [24]:
df1.set_index('timestamp',inplace=True)

In [25]:
sample_cluster_df = df1[df1.cluster_id.isin(list(set(cluster_id_list))[:1000])]

In [26]:
sample_cluster_df['date'] = sample_cluster_df.index.date

/var/folders/bd/srftq4bj7r90mhbkt_4lfx880000gn/T/ipykernel_23748/2049950140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_cluster_df['date'] = sample_cluster_df.index.date


In [ ]:
len(set(sample_cluster_df['cluster_id'].to_list()))

In [29]:
single_day_df = sample_cluster_df.loc[(sample_cluster_df.index.floor('D') == '2022-03-26')]

In [ ]:
two_day_df = df2.loc[(df2.index.floor('D') == '2022-03-26') | (df2.index.floor('D') == '2022-03-27')]

In [30]:
df_single_cluster = single_day_df.loc[single_day_df['cluster_id'] == '01d1e513-9210-4099-967f-6dc13a7b4cc4']

# Total Avg of Cpu
# Min of the cluster cpu usage
# Max of the cluster cpu usage
# Top 5 Min AVG cpu usage
# Top 5 Max AVG cpu usage

In [27]:
def generate_features(current_list):
    #print(current_list)
    current_list.sort()
    metric_max = max(current_list)
    metric_min = min(current_list)
    metric_avg = sum(current_list)/len(current_list)
    top_5_max_avg = sum(current_list[:5])/5
    top_5_min_avg = sum(current_list[-5:])/5
    min_max_diff = metric_max - metric_min
    min_max_5_diff = top_5_max_avg - top_5_min_avg
    #print(metric_max, metric_min, metric_avg, top_5_max_avg, top_5_min_avg, min_max_diff, min_max_5_diff)
    return [ metric_max, metric_min, metric_avg, top_5_max_avg, top_5_min_avg, min_max_diff, min_max_5_diff]
    

In [31]:
sample_cluster_df.head()

,cpu_usage,cluster_id,date
timestamp,,,
2022-03-25 19:30:00,3.878381,032412b9-b016-4989-9974-e18cff9c802f,2022-03-25
2022-03-25 19:45:00,4.774857,032412b9-b016-4989-9974-e18cff9c802f,2022-03-25
2022-03-25 20:00:00,3.936476,032412b9-b016-4989-9974-e18cff9c802f,2022-03-25
2022-03-25 20:15:00,4.205429,032412b9-b016-4989-9974-e18cff9c802f,2022-03-25
2022-03-25 20:30:00,3.828762,032412b9-b016-4989-9974-e18cff9c802f,2022-03-25


In [ ]:
#single day
new_list = []
for cluster_id, current_df1 in single_day.groupby('cluster_id'):
    for date, current_df2 in current_df1.groupby('date'):
        current_list = current_df2['cpu_usage'].to_list()
        features_generated = [cluster_id]
        features_generated.extend(generate_features(current_list))
        new_list.append(features_generated)
    print(cluster_id, count)
new_df = pd.DataFrame(new_list,columns=['cluster_id','metric_max', 'metric_min', 'metric_avg', 'top_5_max_avg', 'top_5_min_avg', 'min_max_diff', 'min_max_5_diff'])
print(count)

In [32]:
#all_days
new_list = []
resampled_df = sample_cluster_df.sample(100)
for cluster_id, current_df1 in sample_cluster_df.groupby('cluster_id'):
        #print(cluster_id,current_df1.head())
        current_list = current_df1['cpu_usage'].to_list()
        features_generated = [cluster_id]
        features_generated.extend(generate_features(current_list))
        new_list.append(features_generated)
new_df2 = pd.DataFrame(new_list,columns=['cluster_id','all_metric_max', 'all_metric_min', 'all_metric_avg', 'all_top_5_max_avg', 'all_top_5_min_avg', 'all_min_max_diff', 'all_min_max_5_diff'])

In [33]:
new_df2.head()

,cluster_id,all_metric_max,all_metric_min,all_metric_avg,all_top_5_max_avg,all_top_5_min_avg,all_min_max_diff,all_min_max_5_diff
0,0118e3df-e1d7-4c8f-8c66-f6be120291c9,12.247143,3.349668,6.245500,4.283667,8.287596,8.897475,-4.003929
1,011c1753-547f-4de1-8906-2b1efd97ab78,5.112952,4.717714,4.915333,1.966133,1.966133,0.395238,0.000000
2,015aac3a-7c20-4295-8d70-21367edb85d4,7.921768,2.274571,5.489657,5.489657,5.489657,5.647196,0.000000
3,01628177-989c-4ee9-903d-8472976dc9ad,16.432976,4.575810,11.684649,9.347719,9.347719,11.857166,0.000000
4,0213ac40-8740-41dd-a688-bb9c6f3550f6,14.847368,3.223429,6.256243,4.229504,7.926085,11.623939,-3.696581


In [10]:
from scipy.stats import zscore

In [11]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=35,min_samples=10).fit(new_df2[['all_metric_max', 'all_metric_min', 'all_metric_avg', 'all_top_5_max_avg', 'all_top_5_min_avg', 'all_min_max_diff', 'all_min_max_5_diff']])

In [6]:
from collections import Counter
import pickle

In [ ]:
labels = db.labels_

In [5]:
Counter(labels)

NameError: name 'labels' is not defined

In [ ]:
with open('current_labels.pkl','wb+') as f:
    pickle.dump(labels,f)

In [7]:
with open('current_labels.pkl','rb') as f:
    new_list2 = pickle.load(f)

In [8]:
Counter(new_list2)

Counter({0: 9924, -1: 76})